In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier  
import sklearn.metrics
rfc = ensemble.RandomForestClassifier()
%matplotlib inline
import time

So here's what you should do. Pick a dataset. It could be one you've worked with before or it could be a new one. Then build the best decision tree you can.

Now try to match that with the simplest random forest you can. For our purposes measure simplicity with runtime. Compare that to the runtime of the decision tree. This is imperfect but just go with it.

In [170]:
# pandas global settings
pd.options.display.max_columns=500
pd.options.display.max_rows=50

In [171]:
path="../../../../Datafiles/titanic/"
file= 'test_complete.csv'
url = 'https://www.kaggle.com/c/titanic/overview'

print("We are working with the Titanic dataset from Kaggle at {}".format(url))
titanic = pd.read_csv(path + file, sep=',', quotechar='"', engine='python')


print("there are {:,} entries in the data frame, with {} columns, from data file: {}".format(len(titanic),
                                                                                             len(titanic.columns),                                                                                            file))
titanic.head(20)
print("titanic has {} rows.".format(len(titanic)))

We are working with the Titanic dataset from Kaggle at https://www.kaggle.com/c/titanic/overview
there are 418 entries in the data frame, with 12 columns, from data file: test_complete.csv
titanic has 418 rows.


In [172]:
titanic.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,0,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,1,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,0,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,1,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,0,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [173]:
# Let's fix some values
titanic["Age"].fillna(titanic["Age"].mean(), inplace=True)
titanic["Fare"].fillna(titanic["Fare"].mean(), inplace=True)

In [174]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,0.363636,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.481622,0.841838,12.634534,0.896760,0.981429,55.840500
min,892.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,0.000000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,0.000000,3.000000,30.272590,0.000000,0.000000,14.454200
75%,1204.750000,1.000000,3.000000,35.750000,1.000000,0.000000,31.500000
max,1309.000000,1.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [175]:
titanic.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,0,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,1,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,0,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,1,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,0,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [224]:
y = titanic['Survived']
#("y is a {}".format(type(y)))
y.columns = ['Survived']
# print(y)

X = titanic.drop(['Survived','Name','Cabin','Sex', 'Ticket' ], axis=1)  # removing Sex results in 57% r-squared


X = pd.get_dummies(X)
# print(X.sample(20))

Why did I need to remove Sex_male, and Sex_female from this dataset?
When I ran it with both Sex_male, and Sex_female, it produced and R-squared of 100%.
When I removed the Sex_male, it still produced an R-squared of 100%.
After removing both of these features, it then provided 

In [225]:
y.head(10)
X.columns

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [226]:
X.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [227]:
X.head(10)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0
3,895,3,27.0,0,0,8.6625,0,0,1
4,896,3,22.0,1,1,12.2875,0,0,1
5,897,3,14.0,0,0,9.2250,0,0,1
6,898,3,30.0,0,0,7.6292,0,1,0
7,899,2,26.0,1,1,29.0000,0,0,1
8,900,3,18.0,0,0,7.2292,1,0,0
9,901,3,21.0,2,0,24.1500,0,0,1


In [228]:
X.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [229]:
classifier = DecisionTreeClassifier() 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)# , random_state=0)  
classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [230]:
y_train.sample(20)

80     0
383    1
342    0
385    1
315    1
197    1
171    0
151    0
175    1
34     0
288    0
154    0
326    1
235    0
377    0
124    0
51     0
360    0
349    1
218    1
Name: Survived, dtype: int64

In [231]:
feature_importances = pd.DataFrame(classifier.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [232]:
feature_importances.head(36)*100

,importance
PassengerId,39.131068
Fare,22.674150
Age,21.504106
SibSp,6.117442
Embarked_Q,4.709591
Pclass,2.425778
Parch,2.122336
Embarked_S,1.315530
Embarked_C,0.000000


In [233]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)# , random_state=0)  
classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [234]:
feature_importances = pd.DataFrame(classifier.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [235]:
feature_importances.head(36)*100

,importance
PassengerId,34.923168
Fare,23.450889
Age,19.171482
SibSp,11.261966
Parch,4.900164
Embarked_Q,4.823610
Pclass,0.691163
Embarked_C,0.388779
Embarked_S,0.388779


In [236]:
y_pred = classifier.predict(X_test)  

In [237]:
start_time = time.time()
# print(sorted(sklearn.metrics.SCORERS.keys()))
cross_val = cross_val_score(classifier, X_test, y_test, cv=10)
print("The mean R2 value for Decision Tree Classifier is {}".format((cross_val.mean())))

The mean R2 value for Decision Tree Classifier is 0.6294444444444445


In [238]:
print("Decision Tree elapsed time --- %s seconds ---" % (time.time() - start_time))

Decision Tree elapsed time --- 0.034639835357666016 seconds ---


In [239]:
X.head(2)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,1


In [240]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})  
df.head(10)

,Actual,Predicted
255,0,0
222,1,0
248,1,1
11,0,0
370,0,0
213,1,1
382,1,0
123,0,0
364,1,1
332,0,0


In [241]:
# print(y_pred)
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))  

Mean Absolute Error: 0.39285714285714285
Mean Squared Error: 0.39285714285714285
Root Mean Squared Error: 0.6267831705280087


In [242]:
rfc = ensemble.RandomForestClassifier(n_estimators=20)

In [243]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)# , random_state=0)  
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [244]:
y_pred = rfc.predict(X_test) 
print("y_pred={}".format(y_pred))

y_pred=[0 0 1 0 1 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0
 0 0 0 1 1 0 0 0 0 0]


In [245]:
start_time = time.time()
cross_val_score(rfc, X_train, y_train, cv=10)

array([0.68571429, 0.62857143, 0.63636364, 0.63636364, 0.66666667,
       0.57575758, 0.63636364, 0.60606061, 0.57575758, 0.51515152])

In [246]:
start_time = time.time()
# print(sorted(sklearn.metrics.SCORERS.keys()))
cross_val2 = cross_val_score(rfc, X_test, y_test, cv=10)
print("The mean R2 value for Random Forest Classification is {}".format((cross_val2.mean())))

The mean R2 value for Random Forest Classification is 0.6069444444444445


In [247]:
print("Random Forest elapsed time --- %s seconds ---" % (time.time() - start_time))

Random Forest elapsed time --- 0.15739107131958008 seconds ---
